In [ ]:
%pip install pandas numpy requests beautifulsoup4 openpyxl

In [9]:
import pandas as pd
import requests
from urllib import response
from bs4 import BeautifulSoup

In [163]:
df = pd.read_excel("data/owner_name.xlsx")
df.head()

,Status,Direct Name,Reverse Name,Record Date Search
0,V,P G A RESORT COMMUNITY PLAT OF GLENWOOD HOMEOW...,LEE JAY,11/10/2025
1,V,BOYNTON LAKES HOMEOWNERS ASSOCIATION NO 2 INC,LOUIS LESLEY JEAN,11/10/2025
2,V,QUADRILLE HOMEOWNERS ASSOCIATION INC,ALEXANDRE DESIMON\nALEXANDRE KERLYNE,11/10/2025
3,V,QUADRILLE HOMEOWNERS ASSOCIATION INC,REYES STEPHEN\nREYES JOAN\nREYES JOSE O,11/10/2025
4,V,SEACREST VILLAS ASSOCIATION INC,HALLER SUSAN,11/10/2025


In [164]:
#cleaning column names (removing leading/trailing spaces)
df.columns = df.columns.str.strip()

In [165]:
df["Reverse Name"] = (
    df["Reverse Name"]
    .astype(str)
    .str.replace(r"\r?\n", " | ", regex=True)
    .str.strip()
)

In [166]:
df["Reverse Name"].head()   
df.loc[3, "Reverse Name"]

'REYES STEPHEN | REYES JOAN | REYES JOSE O'

In [167]:
#helper function to check for empty names

def is_empty_name(name):
    if name is None:
        return True
    if isinstance(name, float):  # NaN case
        return True
    if name.strip() == "":
        return True
    return False

In [168]:
#api call payload builder needs name to look for in the searchText field

def build_payload(name):
    return {
        "inputName": "addresssearch",
        "searchLimit": "20",
        "uID": "89540f28-8b9a-4aed-b609-72529f86a3ca",
        "version": 2,
        "removeZip": True,
        "papaVersion": True,
        "removeChar": "_",
        "removeSpace": True,
        "papaVariance": False,
        "searchText": name
        }
    

In [169]:
#calling api and returning json response
def api_call(name):
    url = "https://maps.pbc.gov/giswebapi/anysearch"
    response = requests.post(url, json=build_payload(name))
    if response.status_code == 200:
        data = response.json()
        print("API call name:", name)
        print("API call data:", data)
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

In [170]:
#finds the pcn number from the api response data for the given address
def get_pcn_numbers(name, data):
    if not data or not name:
        return []

    names = name.split("|")
    all_pcns = []

    for raw_name in names:
        current_name = raw_name.strip().upper()
        print("\nSearching for name:", current_name)

        for item in data:
            search_term = item.get("searchTerm", "").upper()

            # Single contains check
            if current_name in search_term:
                pcn = item.get("PCN")

                if pcn is not None:
                    pcn = str(pcn)   # ← ✅ THIS IS THE ONLY REQUIRED CHANGE

                    print("  Match found, PCN:", pcn)

                    if pcn not in all_pcns:
                        all_pcns.append(pcn)

    return all_pcns



In [171]:
# calling api and getting pcn number for the given address
def get_pcn(name):
    if is_empty_name(name):
        return []

    all_pcns = []

    # Split names BEFORE API call
    names = name.split("|")

    for raw_name in names:
        current_name = raw_name.strip()

        if not current_name:
            continue

        data = api_call(current_name)

        if not data:
            continue

        pcns = get_pcn_numbers(current_name, data)

        for pcn in pcns:
            if pcn not in all_pcns:
                all_pcns.append(pcn)

    return all_pcns



In [191]:
print(df["Reverse Name"].iloc[10])
get_pcn(df["Reverse Name"].iloc[10])

JANKOWSKI TED | JANKOWSKI BARBARA
API call name: JANKOWSKI TED
API call data: [{'searchTerm': 'HAMMAJANG LIMITED | 2906 POLO ISLAND DR, WELLINGTON 33414', 'searchType': 'BUSINESS', 'displayText': 'HAMMAJANG LIMITED | 2906 POLO ISLAND DR, WELLINGTON 33414', 'mapExtent': '-8934140.46173278,3078311.86086184,-8934073.99555828,3078378.52675489', 'lat': 26.6382645475118, 'lng': -80.2566162010727, 'PCN': '73414416090000070', 'dataType': 'search_engine', 'distance': 0.0}, {'searchTerm': 'MANK INCORPORATED | 602 NW 13TH ST 0130, BOCA RATON 33486', 'searchType': 'BUSINESS', 'displayText': 'MANK INCORPORATED | 602 NW 13TH ST 0130, BOCA RATON 33486', 'mapExtent': '-8916737.38142547,3043453.37843952,-8916414.22678689,3043979.40115916', 'lat': 26.3600227037599, 'lng': -80.0990514906235, 'PCN': '0643471922', 'dataType': 'search_engine', 'distance': 0.0}, {'searchTerm': 'FRANKS TEDDY G TR | 1488 NW 48TH LN, BOCA RATON 33431', 'searchType': 'OWNER', 'displayText': 'FRANKS TEDDY G TR | 1488 NW 48TH LN, 

['1242462405']

In [ ]:
df["PCN_List"] = df["Reverse Name"].apply(get_pcn)
df = df.explode("PCN_List").reset_index(drop=True)
df = df.rename(columns={"PCN_List": "PCN"})

In [ ]:
df["PCN"].head()

0    52424210120000127
1    00424729060030160
2                  NaN
3                  NaN
4    26434516240150010
Name: PCN, dtype: object

In [175]:
# function to get property details using pcn number
def get_property_details(pcn):
    if pcn is None or str(pcn).strip() == "":
        return None
    
    url = "https://pbcpao.gov/Property/MapDetails"
    params ={"parcelId": pcn}
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error fetching details for PCN {pcn}: {response.status_code}")
        return None

In [176]:
#getting owners from the property details page
def get_owners(soup):
    owners_section = soup.find("div", class_="map-owners")
    if not owners_section:
        return None

    owners = [
        td.get_text(" ", strip=True)
        for td in owners_section.find_all("td")
        if td.get_text(strip=True)
    ]

    return "; ".join(owners) if owners else None


In [130]:
#getting mailing address from the property details page
def get_mailing_address(soup):
    for row in soup.find_all("tr"):
        label_cell = row.find("td", class_="label")
        if not label_cell:
            continue

        if label_cell.get_text(strip=True) == "Mailing Address":
            value_cell = row.find("td", class_="value")
            if not value_cell:
                return None

            lines = [
                label.get_text(strip=True)
                for label in value_cell.find_all("label")
                if label.get_text(strip=True)
            ]

            return ", ".join(lines) if lines else None

    return None


In [177]:
def get_location(soup):
    for row in soup.find_all("tr"):
        label_cell = row.find("td", class_="label")
        if not label_cell:
            continue

        if label_cell.get_text(strip=True) == "Location":
            value_cell = row.find("td", class_="value")
            if not value_cell:
                return None

            location_label = value_cell.find("label", id="lblLocation")
            if not location_label:
                return None

            return location_label.get_text(strip=True)

    return None

In [178]:
#saving the parsed property details in a dictionary
def parse_property_html(html):
    soup = BeautifulSoup(html, "html.parser")

    return {
        "Owner_Name": get_owners(soup),
        "Mailing_Address": get_mailing_address(soup),
        "Location": get_location(soup)
    }


In [179]:
#creating new columns for Owner_Name and Mailing_Address
df["Owner_Name"] = None
df["Mailing_Address"] = None
df["Location"] = None


for idx, pcn in df["PCN"].items():
    if pd.isna(pcn) or not str(pcn).strip():
        print(f"Skipping row {idx} due to missing PCN")
        continue

    try:
        html = get_property_details(pcn)
        if not html:
            print(f"No HTML returned for PCN {pcn}")
            continue

        parsed = parse_property_html(html)

        df.at[idx, "Owner_Name"] = parsed.get("Owner_Name")
        df.at[idx, "Mailing_Address"] = parsed.get("Mailing_Address")
        df.at[idx, "Location"] = parsed.get("Location")

        print(f"Processed PCN {pcn}")

    except Exception as e:
        print(f"Failed PCN {pcn}: {e}")


Processed PCN 52424210120000127
Processed PCN 00424729060030160
Skipping row 2 due to missing PCN
Skipping row 3 due to missing PCN
Processed PCN 26434516240150010
Processed PCN 00404224000008270
Skipping row 6 due to missing PCN
Skipping row 7 due to missing PCN
Skipping row 8 due to missing PCN
Skipping row 9 due to missing PCN
Error fetching details for PCN 1242462405: 500
No HTML returned for PCN 1242462405
Skipping row 11 due to missing PCN
Processed PCN 38434421151130310
Processed PCN 00424402010001173
Skipping row 14 due to missing PCN
Error fetching details for PCN 5043442604: 500
No HTML returned for PCN 5043442604
Error fetching details for PCN 5043443502: 500
No HTML returned for PCN 5043443502
Error fetching details for PCN 5043442605: 500
No HTML returned for PCN 5043442605
Skipping row 18 due to missing PCN
Skipping row 19 due to missing PCN
Skipping row 20 due to missing PCN
Skipping row 21 due to missing PCN
Skipping row 22 due to missing PCN
Skipping row 23 due to miss

In [180]:
#saving the final dataframe to a new csv file
df.to_csv("data/onwer_name_details.csv", index=False)
